# Dense Document Retriever - Implantação

Ranqueia textos de acordo com um outro texto de entrada através do ranqueador DPR do hugginfaces.<br>
### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [ ]:
%%writefile Model.py
import json
import joblib
import pandas as pd
import numpy as np
from aux_functions import build_result_dataframe



class Model:
    
    def __init__(self):
        self.loaded = False
    
    def load(self):
        artifacts = joblib.load("/tmp/data/sparse_retriever.joblib")
        self.model = artifacts["model"]
        self.question = artifacts["question"]
        self.top = artifacts["top"]
        self.inner_batch_size = artifacts["inner_batch_size"]
        self.columns = artifacts["columns"]
        self.column_context = artifacts["column_context"]
        self.column_score = artifacts["column_score"]
        self.column_doc_id = artifacts["column_doc_id"]
        self.loaded = True

    def predict(self, X, feature_names, meta=None):
        
        if not self.loaded:
            self.load()
            
        if not feature_names:
            feature_names = self.columns
            
            
            
        question = meta["question"] if meta else self.question
        df = pd.DataFrame(X,columns = feature_names)
        report_contexts = list(df[self.column_context])
        
        sim_contexts_ids, scores = self.model(questions=question,
                                     passages = report_contexts,
                                     inner_batch_size = self.inner_batch_size,
                                     top = self.top)
        
        
        df_result = build_result_dataframe(df_input=df,
                            sim_contexts_ids=sim_contexts_ids[0],
                            scores=scores,
                            column_doc_id = self.column_doc_id,
                            column_score = self.column_score)
        
        
        return df.to_numpy() 

In [3]:
# import pandas as pd
# df = pd.read_csv("/tmp/data/fabc-step3-7.csv")
# X = df.to_numpy()

In [5]:
# from Model import Model
# model = Model()
# result = model.predict(X,None)
# result